<a href="https://colab.research.google.com/github/lmontaldo/meli_prueba/blob/Jupyter-Colabs/Similitud_entre_productos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similitud entre productos

In [8]:
! pip install fuzzywuzzy
! pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import requests
import io
import datetime
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import numpy as np
%matplotlib inline
import plotly.express as px

In [10]:
from fuzzywuzzy import process, fuzz
import Levenshtein

In [11]:
! git clone https://github.com/lmontaldo/meli_prueba.git

Cloning into 'meli_prueba'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 37 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [12]:
s='https://raw.githubusercontent.com/lmontaldo/meli_prueba/Datos/items_titles.csv'

In [13]:
simi=pd.read_csv(s,low_memory=False, header=[0],sep=',')

In [14]:
simi.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


Problema: determinar cuán similares son dos títulos del dataset.

In [15]:
simi.shape

(30000, 1)

In [16]:
simi.duplicated().sum()

0

In [17]:
simi.isnull().sum()

ITE_ITEM_TITLE    0
dtype: int64

In [18]:
simi.columns

Index(['ITE_ITEM_TITLE'], dtype='object')

In [19]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
# funcion para remover signos y simbolos
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [21]:
# puntuation
simi['clean']= simi['ITE_ITEM_TITLE'].apply(lambda x:remove_punctuation(x))

In [22]:
# accents
simi['clean']=simi['clean'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [26]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
 def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('portuguese')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [29]:
simi['clean']=simi['clean'].map(lambda s:preprocess(s)) 

In [30]:
simi.head()

,ITE_ITEM_TITLE,clean
0,Tênis Ascension Posh Masculino - Preto E Verme...,tenis ascension posh masculino preto vermelho
1,Tenis Para Caminhada Super Levinho Spider Corr...,tenis caminhada super levinho spider corrida
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,tenis feminino parc hocks blackice original envio
3,Tênis Olympikus Esportivo Academia Nova Tendên...,tenis olympikus esportivo academia nova tenden...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,inteligente led bicicleta tauda luz usb bicicl...


In [31]:
simi['words'] = [len(x.split()) for x in simi['clean'].tolist()]

##  Distancia de Levenshtein



In [32]:
simi.head(9)

,ITE_ITEM_TITLE,clean,words
0,Tênis Ascension Posh Masculino - Preto E Verme...,tenis ascension posh masculino preto vermelho,6
1,Tenis Para Caminhada Super Levinho Spider Corr...,tenis caminhada super levinho spider corrida,6
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,tenis feminino parc hocks blackice original envio,7
3,Tênis Olympikus Esportivo Academia Nova Tendên...,tenis olympikus esportivo academia nova tenden...,7
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,inteligente led bicicleta tauda luz usb bicicl...,8
5,Tênis Casual Masculino Zarato 941 Preto 632,tenis casual masculino zarato preto,5
6,Tênis Infantil Ortopasso Conforto Jogging,tenis infantil ortopasso conforto jogging,5
7,Tv Samsung Qled 8k Q800t Semi Nova,samsung qled semi nova,4
8,Tênis Usthemp Short Temático - Maria Vira-lata 2,tenis usthemp short tematico maria viralata,6


In [33]:
simi.columns

Index(['ITE_ITEM_TITLE', 'clean', 'words'], dtype='object')

In [34]:
nombres_unicos = simi['clean'].unique().tolist()
sorted(nombres_unicos)[:3]

['', 'abotinado infantil novope napa preto', 'acabamento lateral']

In [35]:
nombres_unicos[:5]

['tenis ascension posh masculino preto vermelho',
 'tenis caminhada super levinho spider corrida',
 'tenis feminino parc hocks blackice original envio',
 'tenis olympikus esportivo academia nova tendencia triunfo',
 'inteligente led bicicleta tauda luz usb bicicleta carregave']

In [36]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
t1 = time.time()
print(process.extractOne('tenis feminino parc hocks blackice original envio', nombres_unicos)) 
t = time.time()-t1
print("SELFTIMED:", t)
print("Estimated hours to complete for full dataset:", (t*len(nombres_unicos))/60/60)

('tenis feminino parc hocks blackice original envio', 100)
SELFTIMED: 33.83049511909485
Estimated hours to complete for full dataset: 273.0120956110954


Dado que se precisaría mucho tiempo para completar la tarea, una solución podría ser correr el código por batches o implementar algún otro algoritmo que también sea útil y resuelva nuestro problema. 

Se tomaron los 1000 articulos de la lista y se corrió el algoritmo para ofrecer un resultado parcial de los mismos.

In [37]:
nombres_1000=nombres_unicos[:1000]

In [38]:
# tuple
score_sort = [(x,) + i
             for x in nombres_1000 
             for i in process.extract(x, nombres_1000,  scorer=fuzz.token_sort_ratio)]
#dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['name_sort','match_sort','score_sort'])
similarity_sort.head()

,name_sort,match_sort,score_sort
0,tenis ascension posh masculino preto vermelho,tenis ascension posh masculino preto vermelho,100
1,tenis ascension posh masculino preto vermelho,tenis masculino coca cola xfly happy preto ver...,71
2,tenis ascension posh masculino preto vermelho,tenis casual masculino zarato preto,68
3,tenis ascension posh masculino preto vermelho,tenis rip curl venice masculino pretopreto,67
4,tenis ascension posh masculino preto vermelho,tenis olympikus proof masculino preto vermelho...,66


La distancia de Levenshtein permite medir cuan alejadas estan dos secuencias de palabras. Mide la cantidad mínima de ediciones necesarias en una palabra para que sea igual a otra secuencia. Como se comparan todas las filas, una a una, cuando se cuentan con grandes cantidades de datos ésto puede ser un inconveniete. Por restricciones de tiempo se va a avanzar con la distancia del coseno para completar el ejercicio.

## Distancia del Coseno

In [39]:
import re
# se limpian los strings y se generan los n-grams
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [40]:
simi['clean_grams']=simi['clean'].map(lambda s:ngrams(s))

In [41]:
# matriz TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(nombres_unicos)

La distancia del coseno calcula la similitud entre dos vectores de valores de TF-IDF 
 
Dos desventajas: 
 
 1- La versión de sklearn realiza una gran cantidad de verificación de tipos y manejo de errores.

 2- La versión de sklearn calcula y almacena todas las similitudes de una sola vez, mientras que a nosotros solo nos interesan las más similares.
Por lo tanto, utiliza mucha más memoria de la necesaria.


 Solución: almacenar solo las N coincidencias más altas en cada fila y solo las similitudes por encima de un umbral (opcional)

In [45]:
! pip install sparse_dot_topn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.0 MB/s 


In [47]:
# distancia del coserno

import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [48]:
# top 10 
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 22.509451389312744


In [49]:
# The following code unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'ITE_ITEM_TITLE': left_side,
                          'ITE_ITEM_TITLE ': right_side,
                           'score Similitud (0,1)': similairity})

In [50]:
# resultados
matches_df = get_matches_df(matches, nombres_unicos, top=1000)
matches_df = matches_df[matches_df['score Similitud (0,1)'] < 0.99999] # Remove all exact matches
matches_df.sample(5)

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,"score Similitud (0,1)"
160,tenis infantil bibi fisioflex menino,tenis infantil bibi fisioflex masculino feminino,0.922387
835,tenis cocacola daytona,tenis cocacola shoes daytona,0.841154
289,tenis mizuno wave dynasty feminino,tenis mizuno wave dynasty feminino envio imediato,0.810543
179,tenis feminino casual beira rio,tenis beira rio casual,0.823949
658,tenis masculino olympikus astro marinho,tenis olympikus astro masculino marinho,0.947495


In [51]:
matches_df.sort_values(['score Similitud (0,1)'], ascending=False).head(10)

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,"score Similitud (0,1)"
863,tenis rose confortavel imperdivel,rose confortavel imperdivel,0.965449
798,bicicleta oggi big wheel,bicicleta oggi big wheel aro,0.958751
891,cannondale fsi carbon,bike cannondale fsi carbon,0.953871
73,tenis nike court vision low premium mtz origin...,tenis nike court vision low premium mtz original,0.953164
940,tenis mizuno wave invictus,tenis feminino mizuno wave invictus,0.952792
545,tenis actvitta marinho preto masculino original,tenis actvitta marinho masculino original,0.952776
910,tenis vizzano chunky sneaker confortavel feminino,tenis vizzano chunky sneaker confortavel,0.952575
799,bicicleta oggi big wheel,bicicleta oggi big wheel tam,0.949320
756,tenis rainha drive masculino preto,tenis rainha drive masculino,0.948416
56,tenis vizzano feminino jogging casual,tenis vizzano feminino jogging casual preto,0.947719


Fuente: 

* https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

* https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84

* https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

* https://maxbachmann.github.io/RapidFuzz/process.html